In [6]:
# 각 카테고리의 데이터 개수(채용공고의 개수)를 가져와서 튜플형태로 저장한 뒤, 리스트에 넣는다
category = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17]

category_len = []
for category_num in category:
    data = open('../crawling_all_1211/category{0}.txt'.format(category_num), 'r', encoding = 'utf-8')
    lines = data.readlines()  # 데이터 불러와서 line 단위로 넣는다
    #print(len(lines))
    tu = category_num, len(lines)
    category_len.append(tu)


In [7]:
## 1.
from konlpy.tag import Hannanum
hannanum = Hannanum()

## 2.
stopwords_ko = open('stopwords_ko-Copy1.txt', 'r', encoding='utf-8')
stopwords_ko = stopwords_ko.readlines()
for i in range(len(stopwords_ko)):
    stopwords_ko[i] = stopwords_ko[i].replace('\n', '')
#print(stopwords_ko)

## 3.
import nltk
#nltk.download('stopwords')
stopwords_en = nltk.corpus.stopwords.words('english')
#print(len(stopwords_en), stopwords_en, '\n')

In [8]:
for category_num, data_num in category_len:
    
    data = open('data_cleansing_all/cleansing_category{0}_list{1}.txt'.format(category_num, data_num), 'r', encoding = 'utf-8')
    data = data.readlines()  # 데이터 불러와서 line 단위로 넣는다
    print('#', category_num, '번 카테고리의 크롤링한 채용공고의 개수 = ', len(data))
    
    for i in range(len(data)):
        data[i] = data[i].replace('\n', '')    # 각 문장마다 들어있는 줄바꿈 기호 없애주기
    
    token_list = []
    for i in data: 
        token_list.append(hannanum.morphs(i))  # 리스트 안에 크롤링한 채용공고 개수만큼의 토큰 리스트들이 들어있음
    tokens = sum(token_list, [])               # 이중리스트 해제, 카테고리의 전체 토큰이 저장되어 있습니다.
    
    
    ## 1. pos태깅을 이용한 stopwords 제거
    # 주요 태그 N(체언)과 F(외국어)를 제외한 나머지 단어들을 제거해준다.
    token_pos_list = []
    for i in data:
        token_pos_list.append(hannanum.pos(i)) # 리스트 안에 크롤링한 채용공고 개수만큼의 토큰 리스트들이 들어있음
    tokens_pos = sum(token_pos_list, [])       # 이중리스트 해제, 카테고리의 pos태깅된 전체 토큰이 저장되어 있습니다.
    
    k_tokens = []  # k_tokens에 제거 완료된 토큰을 넣습니다
    pos_stopwords = []
    for i in range(len(tokens_pos)):
        if tokens_pos[i][1] == 'N' or tokens_pos[i][1] == 'F':
            k_tokens.append(tokens[i])
        else:
            pos_stopwords.append(tokens[i])
            
    test = set(pos_stopwords)
    print(len(tokens), '->', len(k_tokens))
    print('#postag를 이용해 불용어를 삭제한 tokens :', test)  # 혹시 모르니까 뭐가 지워졌는지 확인해보기
    
    
    ## 2. 한국어 stopwords 리스트 불러와서 제거
    # 위에서 미리 불러와서 저장해둔 stopwords_ko를 사용한다
    kk_tokens = [w for w in k_tokens if w not in stopwords_ko]   # kk_tokens에 제거 완료된 토큰을 넣습니다
    
    
    ## 3. 영어 stopwords 제거
    # 위에서 미리 불용어 목록 얻음 (소문자화 된 점에 유의)
    k_e_tokens = [w for w in kk_tokens if w.lower() not in stopwords_en]
    # lower()를 이용해서 임시로 소문자화 한 뒤 필터링 한 단어(토큰)들을 k_e_tokens변수에 저장
    
    
    ## 4. 1글자 단어 제거(영어는 제외한다) - 1글자 stopwords 리스트 생성
    # 저장하지 않고 각 카테고리에서 그때그때 만들어 쓴다
    one_word = []  
    for i in k_e_tokens:
        if len(i) == 1:
            #print(i)
            one_word.append(i)  # 1글자 토큰을 one_word 리스트에 저장한다        
    u = set(one_word)  # 유니크 워드만 뽑기
    
    stopwords_one = []
    for i in u:
        if ord('a') <= ord(i.lower()) <= ord('z'):
            print(i)
        else:
            stopwords_one.append(i)
    print(stopwords_one) # 영어를 제외한 1글자 stopwords 리스트 stopwords_one

    final_tokens = [w for w in k_e_tokens if w not in stopwords_one]
    # 필터링 한 단어(토큰)들을 final_tokens변수에 저장
    
    
    ## 전부 소문자로 바꿔서 low_tokens변수에 저장하자
    low_tokens = []
    for i in range(len(final_tokens)):
        low_tokens.append(final_tokens[i].lower())

        
    ## stopwords 제거 완료한 빈도수 추출해보기
    fdistPlain_low = nltk.FreqDist(low_tokens)
    print(fdistPlain_low.most_common(30))  # 빈도수가 높은 토큰 30개 추출
    
    
    ## 각 변수에 저장하자
    print('#최종 token개수: ', len(low_tokens), '\n')
    globals()['tokens{}'.format(category_num)] = low_tokens

# 0 번 카테고리의 크롤링한 채용공고의 개수 =  746
87747 -> 51868
#postag를 이용해 불용어를 삭제한 tokens : {'과도', 'ㄴ가', '어떠하', '잘하', '로써', '빠르', '적', '이상', '없이', '매일', '수시로', '으시어도', '이내', '더라도', '관계없이', '리ㅂ니다', '모르', '생기', '얼마나', '의하', '뛰어넘', '어도', '꼭', '더불', '끊임없이', '더기', '멈추', '향하', '와', '솔직히', '용', '별로', '현재', '준하', '을', 'ㄴ다', '맞', '풀리', '부터', '혹은', '도록', '멋지', '세', '즉시', '높', '배우', '상관없이', '시고', '까다롭', '동적', '를', '들', '어렵', '거리끼', '겠지만', '관하', '도', '놀', '주', '까지의', '길', '떠넘기', '로도', '으면', '로부터', '시거나', '두', '헤', '보다는', '바라', '물', '자는', '라면', '들이', '당장', '함께', '절대', '나타내', '서와의', '었다는', '기대', '새롭', '려', '아거나', '몰리', 'ㅂ니다', '와도', '최근', '이하', '떠나', '는', '기울이', '다르', '능하', '시', '지니', '에만', '갖', '읽', '으로도', '그치', '및', '넘치', '시면', '정적', '으로써', '그러하', '담', '느끼', '실제로', '역시', '통하', '시키', '으면서', '대', '미치', '보', '두렵', '이르', '이러하', '가파르', '발맞추', '로서', '어떻', '끊임없', '먼저', 'ㄹ지', '려고', '여러', '끊', '시ㅁ', '리며', '쉽', '음', '만들', '요', '시는', '이미', '졸', '편하', '선', '파고들', '제하', '대회', '석', '제대로', '따라서', '부담없이', '제반', '이라도', '어야', '받

73906 -> 42449
#postag를 이용해 불용어를 삭제한 tokens : {'과도', 'ㄴ가', '잘하', '짓', '로써', '빠르', '정말', '적', '없이', '이상', '수시로', '으시어도', '이내', '더라도', '관계없이', '리ㅂ니다', '모르', '생기', '얼마나', '랍니다', '의하', '뛰어넘', '어도', '널', '으시다면', '꼭', '더불', '끊임없이', '와', '부르', '빌', '용', '현재', '준하', '을', 'ㄴ다', '맞', '부터', '혹은', '도록', '멋지', '뿐만', '세', '즉시', '높', '배우', '상관없이', '시고', '를', '들', '어렵', '거리끼', '겠지만', '도', '관하', '놀', '주', '까지의', '길', '으면', '로부터', '두', '보다는', '바라', '물', '자는', '라면', '들이', '당장', '함께', '절대', '서와의', '었다는', '기대', '새롭', '려', '아거나', 'ㅂ니다', '별', '와도', '최근', '이하', '떠나', '건', '주되', '는', '기울이', '다르', '시', '능하', '지니', '에만', '읽', '갖', '으로도', '그치', '자주', '막힘없', '오르', '및', '넘치', '시면', '정적', '그러하', '으시더라도', '실제로', '느끼', '담', '역시', '통하', '시키', '대', '보', '두렵', '이르', '이러하', '로서', '방금', '어떻', '끊임없', '가볍', '활발히', 'ㄹ지', '취하', '려고', '여러', '시ㅁ', '리며', '쉽', '음', '만들', '요', '시는', '이미', '졸', '편하', '선', '만큼', '대회', '제대로', '따라서', '부담없이', '이라도', '어야', '받', '알리', 'ㄴ들', '오히려', '목마르', '동시에', '수많', '열리', '계속', '함께하', '신나', '돌보', '하고', '

F
B
G
C
W
E
['혹', '내', '업', '분', '바', '적', '개', '요', '선', '원', '동', '끝', '1', '외', '툴', '기', '義', '호', '용', '소', '유', '팀', '메', '+', '데', '수', '리', '성', '세', '첫', '成', '파', '학', '부', '줄', '2', '도', '전', '뿐', '주', '근', '날', '책', '답', '남', '후', '4', '한', '화', '지', '챗', '차', '3', '중', '문', '초', '–', '뷰', '시', '안', '룰', '드', '삶', '폭', '간', '정', '5', '웹', '양', '백', '린', '위', '대', '미', '으', '仁', '속', '앱']
[('경험', 1045), ('개발', 874), ('이해', 269), ('android', 205), ('서비스', 202), ('사용', 191), ('kotlin', 178), ('경력', 170), ('안드로이드', 154), ('가능', 126), ('코드', 120), ('java', 116), ('ui', 109), ('경험자', 109), ('관심', 99), ('ios', 98), ('api', 94), ('협업', 91), ('3년', 90), ('운영', 88), ('기술', 87), ('mvvm', 84), ('다양한', 81), ('git', 80), ('보유', 80), ('활용', 79), ('모바', 77), ('기반', 76), ('이해도', 75), ('능력', 75)]
#최종 token개수:  14693 

# 5 번 카테고리의 크롤링한 채용공고의 개수 =  274
28588 -> 16612
#postag를 이용해 불용어를 삭제한 tokens : {'과도', 'ㄴ가', '끄', '어떠하', '잘하', '로써', '빠르', '이상', '없이', '으시어도', '이내', '리ㅂ니다', '생기', '얼마나', '어도', '

[('경험', 877), ('데이터', 410), ('개발', 326), ('운영', 237), ('경력', 162), ('분석', 161), ('이해', 148), ('관련', 143), ('서비스', 138), ('구축', 137), ('활용', 131), ('능력', 130), ('python', 124), ('기반', 112), ('사용', 111), ('보유', 111), ('처리', 106), ('aws', 105), ('sql', 101), ('기술', 95), ('시스템', 94), ('경험자', 92), ('가능', 91), ('지식', 83), ('환경', 82), ('역량', 81), ('spark', 80), ('대용량', 74), ('커뮤니케이션', 73), ('db', 71)]
#최종 token개수:  13131 

# 8 번 카테고리의 크롤링한 채용공고의 개수 =  220
24176 -> 14238
#postag를 이용해 불용어를 삭제한 tokens : {'잘하', '짓', '빠르', '이상', '없이', '이내', '더라도', '모르', '얼마나', '랍니다', '꼭', '끊임없이', '와', '솔직히', '용', '현재', '준하', '을', 'ㄴ다', '맞', '부터', '혹은', '도록', '멋지', '세', '높', '배우', '까다롭', '를', '들', '어렵', '거리끼', '도', '관하', '주', '길', '떠넘기', '으면', '두', '헤', '바라', '라면', '들이', '당장', '함께', '서와의', '새롭', '려', '아거나', 'ㅂ니다', '별', '최근', '이하', '는', '다르', '시', '능하', '지니', '갖', '읽', '으로도', '그치', '오르', '및', '넘치', '시면', '정적', '실제로', '느끼', '통하', '시키', '대', '보', '두렵', '이러하', '로서', '어떻', '끊임없', '시ㄴ다면', 'ㄹ지', '여러', '따라잡', '시ㅁ', '리며', '

13107 -> 8975
#postag를 이용해 불용어를 삭제한 tokens : {'타', '내', '꼼꼼하', '롭', '면', '명확히', '나', '크', '빠르', '여러', '정말', '없이', '부탁드리', '이상', '시ㅁ', '계시', '알', '리며', '불', '으로서의', '이내', '음', 'ㅁ', '었거나', '만들', '돕', '요', '시는', '졸', '든지', '제하', '선', '랍니다', '석', '하', '대회', '그', '서', '늘', '어야', '받', '싶', '시기', '사', '의', '보이', '꼭', '기', '끊임없이', '동시에', '열리', '스스로', '모든', '고자', '와', '이나', '같', '피', '현재', '많', '준하', '하고', '넓', '어', '을', '맞', '기이', '또는', '다는', '혹은', '즐기', ';', '거나', '겠습니다', '과의', '세', '즉시', '에서의', 'ㄴ', '높', '배우', '상관없이', '적어도', '접하', '예', '뛰', '대하', '않', '를', '한하', '들', '다', '에서', '쌓', '이', '총', '도', '관하', '서는', '주', '쓰', '원활히', '밝', '짜', '자', '시ㄹ', '두', '맡', '보내', '있', '보다는', '에게', '바라', '에도', '면서', '아', '오', '으시ㄹ', '지', '뜨', '글쓰', '들이', '시ㄴ', '반드시', '다면', '로의', '함께', '모두', '은', 'ㄹ', '참', '이끌', '거치', '으시면', '새롭', '직접', '려', '지키', '와의', '가', 'ㅂ니다', '깊이', '키우', '얻', '겠어', '찾', '마치', '로', '따르', '으시ㄴ', '는', '다르', '능하', '뽑', '폭넓', '시', '없', '에', '갖추', '좋', '더', '묻', '읽', '까지', '갖', '간', '바꾸', '어서',

13320 -> 8495
#postag를 이용해 불용어를 삭제한 tokens : {'타', '내', '롭', '면', '명확히', '이루', '여러', '나', '빠르', '크', '이상', '계시', '알', '개', 'ㅁ', '음', '더라도', '살피', '만들', '멀', '요', '시는', '만나', '제하', '꾸준히', '선', '서의', '석', '하', '대회', '그', '서', '늘', '어야', '받', '싶', '어서ㄴ', '사', '의', '으시다면', '기', '끊임없이', '함께하', '열리', '스스로', '고자', '모든', '와', '배', '이나', '용', '같', '피', '많', '준하', '하고', '을', '어', '즐겁', '맞', '임하', '비', '기이', '리', '또는', '서로', '부터', '혹은', '즐기', '거나', '과의', '에서의', 'ㄴ', '높', '배우', '시고', '예', '뛰', '대하', '않', '를', '들', '어렵', '다', '에서', '쌓', '이', '총', '거리끼', '도', '작', '놀', '었는지', '주', '까지의', '아우르', '쓰', '원활히', '밝', '메이', '아낌없이', 'ㄴ다는', '으면', '겪', '자', '로부터', '시ㄹ', '있', '보다는', '에게', '바라', '물', '면서', '아', '아니', '오', '지', '시ㄴ', '다면', '함께', '모두', '은', 'ㄹ', '서와의', '이끌', '새롭', '직접', '려', '아거나', '와의', '가', '가장', 'ㅂ니다', '찾', '건', '보다', '그렇', '로', '으시ㄴ', '는', '폭넓', '안', '시', '에', '없', '더', '갖추', '좋', '지니', '갖', '읽', '셨거나', '까지', '간', '어서', '특히', '및', '게', '과', 'ㄴ단', '는지', '원하', '다루', '재미있', '다른', '실제로', '느끼', '만

In [6]:
#for문을 이용해 모든 tokens변수들을 저장해줍시다 (임시입니다, 변경될 수 있습니다)

for i in category:
    tokens = globals()['tokens{}'.format(i)]
    line = '\n'.join(tokens)
    
    f = open('data_stopwords_filtering_all/tokens{}.txt'.format(i), 'w')
    f.write(line)
    f.close() 

In [18]:
# 빈도수 top5 찍어보기

for category_num in range(18):
    tokens = globals()['tokens{}'.format(category_num)]
    print('tokens{} :'.format(category_num), nltk.FreqDist(tokens).most_common(5))
    
## 필요 이상으로 많이 등장하는 쓸모없는 단어들을 삭제해줘야 합니다.
over_word = ['경험', '개발', '서비스', '이해', '운영', '사용', '경험자', '테스트', '보유', '관련', '경력', '데이터', '구축']

tokens0 : [('경험', 3259), ('개발', 2127), ('서비스', 813), ('이해', 700), ('운영', 698)]
tokens1 : [('경험', 2540), ('개발', 1962), ('이해', 742), ('서비스', 554), ('사용', 522)]
tokens2 : [('경험', 2528), ('개발', 1825), ('이해', 786), ('사용', 535), ('서비스', 448)]
tokens3 : [('경험', 1996), ('개발', 1470), ('이해', 457), ('서비스', 416), ('java', 401)]
tokens4 : [('경험', 1045), ('개발', 874), ('이해', 269), ('android', 205), ('서비스', 202)]
tokens5 : [('경험', 984), ('개발', 803), ('ios', 298), ('이해', 277), ('사용', 202)]
tokens6 : [('경험', 994), ('개발', 554), ('python', 211), ('이해', 210), ('서비스', 205)]
tokens7 : [('경험', 877), ('데이터', 410), ('개발', 326), ('운영', 237), ('경력', 162)]
tokens8 : [('경험', 917), ('개발', 588), ('이해', 275), ('서비스', 215), ('사용', 194)]
tokens9 : [('경험', 842), ('운영', 334), ('개발', 249), ('서비스', 187), ('구축', 177)]
tokens10 : [('경험', 539), ('데이터', 353), ('분석', 232), ('관련', 163), ('개발', 158)]
tokens11 : [('경험', 518), ('운영', 255), ('개발', 157), ('경력', 128), ('구축', 125)]
tokens12 : [('개발', 464), ('경험', 431), ('경험자', 121), ('관

In [19]:
## vocabulary_all_1211 디렉토리의 least_word와 같이 제거해주기 위해 저장합니다
line = '\n'.join(over_word)
f = open('../vocabulary_all_1211/over_word.txt'.format(i), 'w')
f.write(line)
f.close() 